In [2]:
!pip install imblearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
# sklearn packages (for pre-processing, model-building, evaluators, cross-validators, etc.)
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, PowerTransformer
from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, recall_score, precision_score, classification_report, roc_auc_score, f1_score
from sklearn.metrics import precision_recall_curve, auc
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import OneHotEncoder

# Imblearn packages (for over_sampling and imblearn pipelines)
from imblearn import over_sampling
from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler
from imblearn.pipeline import Pipeline, make_pipeline  ## Using packages from imblearn.pipeline for making pipelines

## xgboost packages
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance

!pip install tqdm
from tqdm import tqdm

from random import sample

In [3]:
# create 10 data sets according to time, proportion of frauds, and (cluster?)
df_cct = pd.read_parquet("/home/onyxia/work/df_cct_stand2.parquet")
df_amount = pd.read_parquet("/home/onyxia/work/df_all.parquet")


ArrowInvalid: Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.

In [4]:
df_amount = pd.read_parquet("/home/onyxia/work/df_all.parquet")


In [5]:
df_amount

,User,Card,Month,Day,Hours,year_diff,Amount,Use Chip,Card Brand,Card Type,...,daily_amount,nb_daily_declines_card,nb_monthly_declines_card,bad_pin,insufficient_balance,hr_nbt/last_30d_av_hr_nbt,day_nbt/last_30d_av_day_nbt,last_3d_amt/nbt,label_global,Is Fraud?
0,0,0,9,1,6,-6,134.09,Swipe Transaction,Visa,Debit,...,172.57,0,0.0,0,0,1.000000,1.000000,134.090000,0,0
1,0,0,9,1,6,-6,38.48,Swipe Transaction,Visa,Debit,...,172.57,0,0.0,0,0,1.000000,1.000000,86.285000,0,0
2,0,0,9,2,6,-6,120.34,Swipe Transaction,Visa,Debit,...,249.29,0,0.0,0,0,0.600000,1.000000,97.636667,0,0
3,0,0,9,2,17,-6,128.95,Swipe Transaction,Visa,Debit,...,249.29,0,0.0,0,0,0.666667,1.000000,105.465000,0,0
4,0,0,9,3,6,-6,104.71,Swipe Transaction,Visa,Debit,...,190.90,0,0.0,0,0,0.714286,1.000000,105.314000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24386895,1999,1,2,27,22,3,-54.00,Chip Transaction,Visa,Credit,...,423.71,0,1.0,0,0,2.368421,1.568627,62.315294,0,0
24386896,1999,1,2,27,22,3,54.00,Chip Transaction,Visa,Credit,...,423.71,0,1.0,0,0,2.341935,1.561290,61.853333,0,0
24386897,1999,1,2,28,7,3,59.15,Chip Transaction,Visa,Credit,...,147.40,0,1.0,0,0,0.780645,0.586430,52.769286,0,0
24386898,1999,1,2,28,20,3,43.12,Chip Transaction,Visa,Credit,...,147.40,0,1.0,0,0,0.779221,0.582524,52.126000,0,0


In [ ]:
results_df = pd.DataFrame(columns = ["param", "our_score_cv","min_our_score_cv", 
"max_our_score_cv", "f1_cv", "precision_cv", "recall_cv"])

In [ ]:
# Manual gridsearch
grid_search_dicts =  [dict(zip(['class_weight', "n_estimators",'max_depth', 'min_samples_leaf'], [a, b, c, d]))
for a, b, c, d in itertools.product([{1:i, 0:1} for i in [10, 15, 20]], [10, 50, 100, 200, 300], [1, 5, 10, 15, 20], [1, 5, 10, 25, 50, 100, 200])]


for i in range(len(grid_search_dicts)): #???
    our_score = []
    f1 = []
    recall = []
    precision = []
    for j in range():
        model = RandomForestClassifier(**grid_search_dicts[i])
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        our_score.append(our_score(model, X_test, y_test))
        f1.append(f1_score(y_test, y_pred))
        recall.append(recall_score(y_test, y_pred))
        precision.append(precision_score(y_test, y_pred))
    results_df.loc[len(results_df)] = [grid_search_dicts[i], our_score.mean(), 
    our_score.min(), our_score.max(), f1.mean(), precision.mean(), recall.mean()]


        
